In [63]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Font, Side, Border, numbers

In [64]:
col_list = [
    ["A", "고객사", 20],
    ["B", "서비스 일자", 12],
    ["C", "서비스 내역", 40],
    ["D", "서비스 기사", 12],
    ["E", "유상/무상", 10],
    ["F", "공급가액(원)", 15],
    ["G", "부가세(원)", 15],
    ["H", "합계(원)", 15]
]

In [65]:
def makeInvoiceForm(ws):
    ws["A2"] = "(주)카피맨 렌탈 복사기 서비스 비용 청구서"
    ws["A4"] = "※서비스 기간 : 2021년 10월"

    for col in col_list:
        ws[col[0] + "5"] = col[1]

    ws["A8"] = "위 금액을 청구합니다."
    ws["A9"] = "2021년 10월 29일"
    ws["A10"] = "(주)카피맨 대표이사 김카피(인)"
    return ws

In [66]:
def stylizeInvoiceForm(ws, item_start_row_no, item_row_no):
    side = Side(color="CCCCCC", border_style="medium")
    border_style = Border(
        left = side,
        right = side,
        top = side,
        bottom = side,
    )

    for col in col_list:
        ws.column_dimensions[col[0]].width = col[2]
        ws[col[0] + "5"].font = Font(sz="12", bold=True)
        ws[col[0] + "5"].fill = PatternFill(patternType="solid", start_color="ffd663")
        ws[col[0] + "5"].border = border_style
        ws[col[0] + "5"].alignment = Alignment(horizontal = "center", vertical = "center")
        for row_no in range(item_start_row_no, item_row_no+1):
            ws["{}{}".format(col[0], row_no)].font = Font(sz="11", bold=False)
            ws["{}{}".format(col[0], row_no)].border = border_style
            if col[0] in ["A", "B", "C", "D", "E"]:
                ws["{}{}".format(col[0], row_no)].alignment = Alignment(horizontal = "center", vertical = "center")
            else:
                ws["{}{}".format(col[0], row_no)].alignment = Alignment(horizontal = "right", vertical = "center")
                ws["{}{}".format(col[0], row_no)].number_format = numbers.BUILTIN_FORMATS[37]
        ws["{}{}".format(col[0], item_row_no)].font = Font(bold=True)
        ws["{}{}".format(col[0], item_row_no)].fill = PatternFill(patternType="solid", start_color="eeeeee")

    ws.merge_cells("A2:H2")
    ws.merge_cells("A4:H4")
    ws.merge_cells("B{}:E{}".format(item_row_no, item_row_no))
    ws.merge_cells("A{}:H{}".format(item_row_no+2, item_row_no+2))
    ws.merge_cells("A{}:H{}".format(item_row_no+3, item_row_no+3))
    ws.merge_cells("A{}:H{}".format(item_row_no+4, item_row_no+4))

    ws["A2"].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A2"].font = Font(sz="20", bold=True)
    
    ws["A4"].alignment = Alignment(horizontal = "right", vertical = "center")
    
    ws["A{}".format(item_row_no+2)].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+2)].font = Font(sz="16", bold=True)
    
    ws["A{}".format(item_row_no+3)].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+3)].font = Font(sz="12", bold=True)
    
    ws["A{}".format(item_row_no+4)].alignment = Alignment(horizontal = "right", vertical = "center")
    ws["A{}".format(item_row_no+4)].font = Font(sz="14", bold=True)

    ws.row_dimensions[2].height = 40
    ws.row_dimensions[4].height = 20
    ws.row_dimensions[5].height = 25
    ws.row_dimensions[item_row_no+2].height = 40
    ws.row_dimensions[item_row_no+3].height = 20
    ws.row_dimensions[item_row_no+4].height = 40

    for row_no in range(item_start_row_no, item_row_no+1):
        ws.row_dimensions[row_no].height = 25
        
    return ws

In [67]:
df_src = pd.read_excel("sample_data/서비스 이력.xlsx", sheet_name = None) #모든 시트를 하나의 df로 로드
df = pd.concat(df_src, ignore_index=True)
customer_list = df["고객사명"].unique()

for customer in customer_list:
    wb = Workbook()
    wb["Sheet"].title = "서비스 청구서"
    ws = makeInvoiceForm(wb["서비스 청구서"])
    item_row_no = 6
    item_start_row_no = item_row_no
    this_customer_items = df.loc[df["고객사명"] == customer]
    this_customer_items = this_customer_items.sort_values(by=["방문일자", "기사"])
    for idx in range(len(this_customer_items)):
        ws.insert_rows(item_row_no)
        ws["A{}".format(item_row_no)] = this_customer_items.iloc[idx, 1]
        ws["B{}".format(item_row_no)] = this_customer_items.iloc[idx, 0]
        ws["C{}".format(item_row_no)] = this_customer_items.iloc[idx, 3]
        ws["D{}".format(item_row_no)] = this_customer_items.iloc[idx, 2]
        ws["E{}".format(item_row_no)] = this_customer_items.iloc[idx, 4]
        ws["F{}".format(item_row_no)] = this_customer_items.iloc[idx, 5]
        ws["G{}".format(item_row_no)] = this_customer_items.iloc[idx, 6]
        ws["H{}".format(item_row_no)] = this_customer_items.iloc[idx, 7]
        item_row_no += 1
    ws["A{}".format(item_row_no)] = "총계"
    ws["B{}".format(item_row_no)] = '=COUNTA(C{}:C{})&"건"'.format(item_start_row_no, item_row_no-1)
    ws["F{}".format(item_row_no)] = "=SUM(F{}:F{})".format(item_start_row_no, item_row_no-1)
    ws["G{}".format(item_row_no)] = "=SUM(G{}:G{})".format(item_start_row_no, item_row_no-1)
    ws["H{}".format(item_row_no)] = "=SUM(H{}:H{})".format(item_start_row_no, item_row_no-1)
    
    ws = stylizeInvoiceForm(ws, item_start_row_no, item_row_no)
    
    wb.save("sample_data/invoce_results/서비스 청구서_"+this_customer_items.iloc[idx, 1]+".xlsx")
    print("청구서 생성완료 : {}".format(this_customer_items.iloc[idx, 1]))

청구서 생성완료 : 이주그룹
청구서 생성완료 : 문거창고
청구서 생성완료 : 육대장
청구서 생성완료 : 영호몰딩
청구서 생성완료 : 주한제분
청구서 생성완료 : 룹부제철
청구서 생성완료 : 삼도네트웍스
청구서 생성완료 : 한건도로공사
청구서 생성완료 : 힐원무역
청구서 생성완료 : 동튜디오드래곤
청구서 생성완료 : 아울코닉스 엔터테인먼트
청구서 생성완료 : 대킨푸드
청구서 생성완료 : 아호타이어
청구서 생성완료 : 선어
청구서 생성완료 : 항진그룹
청구서 생성완료 : 한산그룹
청구서 생성완료 : 아대캐피탈
청구서 생성완료 : 로라면세점
청구서 생성완료 : 윕노컴퍼니
청구서 생성완료 : 피일대우버스
청구서 생성완료 : 엔스소프트
청구서 생성완료 : 삼열SDS
청구서 생성완료 : 더수랙레이블
청구서 생성완료 : 나선스평가정보
청구서 생성완료 : 업랜드백화점
청구서 생성완료 : 신삼계
청구서 생성완료 : 방이소아성산업
청구서 생성완료 : 아근카인텍
청구서 생성완료 : 오대중공업그룹
청구서 생성완료 : 옥대
청구서 생성완료 : 라오라패션디자인전문학교
청구서 생성완료 : 삼롯시멘트
청구서 생성완료 : 협농공업사
청구서 생성완료 : 삼룹그룹
청구서 생성완료 : 대공씨아이
청구서 생성완료 : 애업그룹
청구서 생성완료 : 중루토건
청구서 생성완료 : 성성생명
청구서 생성완료 : 일데하이마트
청구서 생성완료 : 한트케미칼
청구서 생성완료 : 자양증권
청구서 생성완료 : 파엔터테인먼트
청구서 생성완료 : 삼디사
청구서 생성완료 : 발데지알에스
청구서 생성완료 : 타오롱글로벌
청구서 생성완료 : 동석제약
청구서 생성완료 : 웅나식품
청구서 생성완료 : 국두투어
청구서 생성완료 : 파노마걸
청구서 생성완료 : 래주은행
청구서 생성완료 : 세사그룹
청구서 생성완료 : 그랩
청구서 생성완료 : 공진공업사
청구서 생성완료 : 펜콩단잠
청구서 생성완료 : 한크일보
청구서 생성완료 : 대스전자부품
청구서 생성완료 : 한룹공영
청구서 생성완료 : 삼전식품
청구서 생성완료 : 중스코
청구서 생